In [5]:
pip install azure-eventhub 


StatementMeta(, 97944be0-9163-4fd4-9f10-bf697a99bf29, 7, Finished, Available, Finished)

Note: you may need to restart the kernel to use updated packages.


In [6]:
import requests
import json
import time
from datetime import datetime
from datetime import datetime, timezone, timedelta
from azure.eventhub import EventHubProducerClient, EventData
import notebookutils



# -------------------------------
# Configuration
# -------------------------------
API_KEY = "b97479a794fa4c94b9c64911252710"
IST = timezone(timedelta(hours=5, minutes=30))
points = {
    "Connaught Place": (28.6139, 77.2090),
    "Noida": (28.5355, 77.3910),
    "West Delhi": (28.7041, 77.1025),
    "Gurugram": (28.4595, 77.0266),
    "New Delhi":(28.633,77.222)
}
#WEATHER_URL = f"https://api.weatherapi.com/v1/current.json?q={LAT},{LON}&key={API_KEY}&aqi=yes"

EVENTHUB_CONNECTION_STR = "Endpoint=sb://esehbl6c19ci24lun7kr7q.servicebus.windows.net/;SharedAccessKeyName=key_64345f68-80f8-4ef8-8d80-a1ea2cadba82;SharedAccessKey=5cTwtV1oWhs4qdzYy3g/vkQgbrdDZKoU9+AEhDnm3dg=;EntityPath=es_648849a4-b428-4a34-ba8c-1a1be10efe93"
EVENTHUB_NAME = "weatherapi"
producer = EventHubProducerClient.from_connection_string(conn_str=EVENTHUB_CONNECTION_STR)

# Get the current datetime object
now = datetime.now()
# Convert to Unix timestamp (float, seconds since epoch)
timestamp_seconds = now.timestamp()
# Convert to milliseconds and cast to an integer
timestamp_millis = int(timestamp_seconds * 1000)
print(timestamp_millis)
dt_str = datetime.now(IST).isoformat()
dt_obj = datetime.fromisoformat(dt_str)
formatted_dt = dt_obj.strftime("%Y-%m-%d %H:%M")
#run_id = mssparkutils.pipeline.getPipelineRunId()
# -------------------------------
# Fetch weather data
# -------------------------------
def fetch_weather(location_name, lat, lon):
    url = f"https://api.weatherapi.com/v1/current.json?q={lat},{lon}&key={API_KEY}&aqi=yes"
    try:
        response = requests.get(url)
        response.raise_for_status()
        data = response.json()

        # Flatten JSON to match schema
        event = {
            "location": data["location"],         
            "current": data["current"],
            "city": data["location"]["name"],           
        }
        print(f"✅ Fetched weather for {location_name} ({lat}, {lon})")
        
        return event
    except requests.RequestException as e:
        print(f"❌ Error fetching weather for {location_name}: {e}")
        return None

# -------------------------------
# Send batch of events to Event Hub
# -------------------------------
def send_to_eventhub(events):
    try:
        with producer:
            event_data_batch = producer.create_batch()
            for event in events:
                event_data_batch.add(EventData(json.dumps(event)))
            producer.send_batch(event_data_batch)
        print(f"📤 Sent {len(events)} events to Event Hub at {datetime.utcnow().isoformat()}")
    except Exception as e:
        print(f"❌ Error sending to Event Hub: {e}")

all_events = []

for name, (lat, lon) in points.items():
        event = fetch_weather(name, lat, lon)
        if event:
            all_events.append(event)
            time.sleep(2)  # small delay between API calls to avoid throttling
if all_events:
        send_to_eventhub(all_events)
        print(all_events)

  


StatementMeta(, 97944be0-9163-4fd4-9f10-bf697a99bf29, 8, Finished, Available, Finished)

1769490433804
✅ Fetched weather for Connaught Place (28.6139, 77.209)
✅ Fetched weather for Noida (28.5355, 77.391)
✅ Fetched weather for West Delhi (28.7041, 77.1025)
✅ Fetched weather for Gurugram (28.4595, 77.0266)
✅ Fetched weather for New Delhi (28.633, 77.222)
📤 Sent 5 events to Event Hub at 2026-01-27T05:07:25.073667
[{'location': {'name': 'New Delhi', 'region': 'Delhi', 'country': 'India', 'lat': 28.6, 'lon': 77.2, 'tz_id': 'Asia/Kolkata', 'localtime_epoch': 1769490356, 'localtime': '2026-01-27 10:35'}, 'current': {'last_updated_epoch': 1769490000, 'last_updated': '2026-01-27 10:30', 'temp_c': 15.2, 'temp_f': 59.4, 'is_day': 1, 'condition': {'text': 'Moderate or heavy rain with thunder', 'icon': '//cdn.weatherapi.com/weather/64x64/day/389.png', 'code': 1276}, 'wind_mph': 8.1, 'wind_kph': 13.0, 'wind_degree': 155, 'wind_dir': 'SSE', 'pressure_mb': 1019.0, 'pressure_in': 30.09, 'precip_mm': 0.06, 'precip_in': 0.0, 'humidity': 63, 'cloud': 75, 'feelslike_c': 15.2, 'feelslike_f': 5